<a href="https://colab.research.google.com/github/astrapi69/DroidBallet/blob/master/SQL_with_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><a target="_blank" href="https://academy.constructor.org/"><img src="https://jobtracker.ai/static/media/constructor_academy_colour.b86fa87f.png" width="200" style="background:none; border:none; box-shadow:none;" /></a> </center>

_____
<center> <h1>  </h1> </center>
<center> <h1> Advacend SQL </h1> </center>

<p style="margin-bottom:1cm;"></p>

_____

<center>Constructor Academy, 2024</center>

# Connecting to a database with Python

To be able to extract data and manipulate a database with python, we first import two packages:

In [ ]:
import sqlalchemy as db
import pandas as pd
from sqlalchemy import text

In [ ]:
db.__version__

[SQLalchemy](https://www.sqlalchemy.org) is a powerful library that allows python coders to interact with a SQL database. In Combination with Pandas, it allows you to query and modify databases and to  get the results of queries in the convenient form of a dataframe.

First, we connect to the database (library.db) which we created in SQL Day 1:

In [ ]:
engine = db.create_engine('sqlite:///content/library.db', echo=True)

connection string can vary depending on your operating system. check [documentation](https://docs.sqlalchemy.org/en/14/dialects/sqlite.html#connect-strings)

You need to specify the relative path of the SQLite database file after the `///`.
We use the option `echo=True`, which will display the SQL query used by SQLAlchemy.

We can display the list of tables in this database using:

In [ ]:
engine

In [ ]:
inspector = db.inspect(engine)
inspector.get_table_names()

2021-10-11 21:34:24,708 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-10-11 21:34:24,710 INFO sqlalchemy.engine.base.Engine ()
2021-10-11 21:34:24,712 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-10-11 21:34:24,713 INFO sqlalchemy.engine.base.Engine ()
2021-10-11 21:34:24,714 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-10-11 21:34:24,715 INFO sqlalchemy.engine.base.Engine ()


['Author',
 'Book',
 'Book_format',
 'Book_theme',
 'Borrow',
 'Department',
 'Employee',
 'Format',
 'Theme',
 'User',
 'sqlite_sequence']


For a given table, we can also display information on columns. Let's Check out the User table:

In [ ]:
pd.DataFrame(inspector.get_columns('User'))

2021-10-11 21:59:45,771 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("User")
2021-10-11 21:59:45,772 INFO sqlalchemy.engine.base.Engine ()


,name,type,nullable,default,autoincrement,primary_key
0,id,INTEGER,True,None,auto,1
1,first_name,TEXT,False,None,auto,0
2,last_name,TEXT,False,None,auto,0
3,email,TEXT,False,None,auto,0
4,phone,TEXT,True,None,auto,0
5,address,TEXT,True,None,auto,0
6,zip,TEXT,True,None,auto,0
7,city,TEXT,False,None,auto,0
8,description,TEXT,True,None,auto,0
9,created,DATETIME,True,None,auto,0


# SQL Commands

We can execute any SQL queries using the execute method of the connection object. Let's say we want to examine the first row of the User table. The command is:

In [ ]:
con = engine.connect()
rows = con.execute(text('SELECT * FROM User;'))
for row in rows:
  print(row)
con.close()

In [ ]:
with engine.connect() as con:
    rows = con.execute(text('SELECT * FROM User LIMIT 1;'))
    for row in rows:
        print(row)

2021-10-11 21:59:51,636 INFO sqlalchemy.engine.base.Engine SELECT * FROM User LIMIT 1;
2021-10-11 21:59:51,637 INFO sqlalchemy.engine.base.Engine ()
(12, 'Manu', 'Ginobili', 'el@manu.ch', None, 'Bahnhofstrasse', '6000', 'Luzern', None, '2004-03-11')


Let's dissect the command that we just executed:

- **First,** we opened a connection with the database engine:
```python
with engine.connect() as con:
```
We use a `with ... as` statement, so that the connection is closed at the end of the block.
- **Then,** we use the `execute` method on the connection object `con`:
```python
con.execute(SQL_string)
```
with `SQL_string` the SQL command to display the first row of the Employee table:
```SQL
SELECT * FROM Employee LIMIT 1;
```
This statement queries the values of all the columns of the employee table, for the first row.

**USER Interaction**

In [ ]:
name_ = "Manu"
with engine.connect() as con:
    rows = con.execute(text(f'SELECT * FROM User WHERE first_name = "{name_}";'))
    print(rows)
    for row in rows:
        print(row)

2021-10-11 22:00:04,101 INFO sqlalchemy.engine.base.Engine SELECT * FROM User WHERE first_name = ?;
2021-10-11 22:00:04,102 INFO sqlalchemy.engine.base.Engine ('Manu',)
(12, 'Manu', 'Ginobili', 'el@manu.ch', None, 'Bahnhofstrasse', '6000', 'Luzern', None, '2004-03-11')


In [ ]:
values = {'name': "Manu"}
stmt = text('SELECT * FROM User WHERE first_name = :name')
with engine.connect() as con:
    rows = con.execute(stmt, values)
    for row in rows:
        print(row)

2021-10-11 22:00:04,101 INFO sqlalchemy.engine.base.Engine SELECT * FROM User WHERE first_name = ?;
2021-10-11 22:00:04,102 INFO sqlalchemy.engine.base.Engine ('Manu',)
(12, 'Manu', 'Ginobili', 'el@manu.ch', None, 'Bahnhofstrasse', '6000', 'Luzern', None, '2004-03-11')


### INSERT

To insert a new User:

In [ ]:
with engine.connect() as con:
    res = con.execute(text('''
        INSERT INTO User (first_name , last_name, email, city)
        VALUES
             ('John', 'Doe', 'blabla@gmail.com', 'Bern'),
             ('Barack',  'Obama', 'blablabla@gmail.com', 'Basel');'''))
    # con.commit()

2021-05-28 14:40:57,631 INFO sqlalchemy.engine.base.Engine 
        INSERT INTO Customer (FirstName , LastName, Email)
        VALUES 
             ('John', 'Doe', 'blabla@gmail.com'),
             ('Barack',  'Obama', 'blablabla@gmail.com');
2021-05-28 14:40:57,632 INFO sqlalchemy.engine.base.Engine ()
2021-05-28 14:40:57,633 INFO sqlalchemy.engine.base.Engine COMMIT


- Let's check that Barack Obama is in the database:

In [ ]:
with engine.connect() as con:
    res = con.execute(text('SELECT * FROM User WHERE first_name = "Barack";'))
    con.commit()
    for row in res:
        print(row)

2021-05-28 14:41:02,408 INFO sqlalchemy.engine.base.Engine SELECT * FROM Customer WHERE FirstName == "Barack";
2021-05-28 14:41:02,410 INFO sqlalchemy.engine.base.Engine ()
(61, 'Barack', 'Obama', None, None, None, None, None, None, None, None, 'blablabla@gmail.com', None)


Here you can see that a CustomerID was set automatically for the new customer Barack Obama.

### UPDATE

If we want to update data for a row that already exists, we can simply use the `UPDATE` keyword:

In [ ]:
barrack_id = row[0]

In [ ]:
with engine.connect() as con:
    res = con.execute(text('''
        UPDATE User
        SET email = 'barack.obama@gmail.com'
        WHERE
        id = :id_;'''), ({"id_": barrack_id}))
    # con.commit()

2021-05-28 14:41:07,797 INFO sqlalchemy.engine.base.Engine 
        UPDATE Customer
        SET Email = 'barack.obama@gmail.com'
        WHERE
        CustomerId = 63;
2021-05-28 14:41:07,798 INFO sqlalchemy.engine.base.Engine ()
2021-05-28 14:41:07,799 INFO sqlalchemy.engine.base.Engine COMMIT


We can check that the row has been updated:

In [ ]:
with engine.connect() as con:
    res = con.execute(text('SELECT * FROM User WHERE first_name = "Barack";'))
    for row in res:
        print(row)

2021-05-28 14:41:13,160 INFO sqlalchemy.engine.base.Engine SELECT * FROM Customer WHERE FirstName == "Barack";
2021-05-28 14:41:13,162 INFO sqlalchemy.engine.base.Engine ()
(61, 'Barack', 'Obama', None, None, None, None, None, None, None, None, 'blablabla@gmail.com', None)


### DELETE

Finally, we can delete a row using the `DELETE FROM` keywords:

In [ ]:
id_ = ???
with engine.connect() as con:
    res = con.execute(text(f'''
        DELETE FROM User
        WHERE id = "{id_}";'''))
    # con.commit()

2021-05-28 14:41:23,816 INFO sqlalchemy.engine.base.Engine 
        DELETE FROM Customer
        WHERE CustomerID==63;
2021-05-28 14:41:23,818 INFO sqlalchemy.engine.base.Engine ()
2021-05-28 14:41:23,821 INFO sqlalchemy.engine.base.Engine COMMIT


# Query/write data using SQL through pandas

As you can see from the last section on `SELECT`, the output of the SQL query is not so convenient, and particularly, if you are performing data analysis, it might be better for you to directly read/write to a SQL database from pandas dataframes. Be aware, however, that this method becomes very slow for large amounts of data.

Pandas provides 2 main functions to interact with a database:

- `read_sql_query` ([see documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_sql_query.html)), that outputs a dataframe from a sql query.

The syntax is the following:

```python
df = pd.read_sql_query('SQL query', connection)
```
    - 'SQL query' is a string containing the SQL query
    - connection is the connected engine object that we defined earlier (con)
    - we obtain df, a dataframe containing the result of the query


- `to_sql` ([see documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_sql.html)), which is a dataframe method that writes the dataframe into a SQL database.

The syntax is the following:

```python
df.to_sql('name', connection)
```
    - name is the name of the table in the SQL database
    - connection is the SQLalchemy connection object


## read_sql_query
Let's try it on the User table:

In [ ]:
with engine.connect() as con:
    df = pd.read_sql_query(text("""
    SELECT *
    FROM User;
        """), con)
df.head()

2021-10-11 22:04:46,458 INFO sqlalchemy.engine.base.Engine 
    SELECT *
    FROM User;
        
2021-10-11 22:04:46,459 INFO sqlalchemy.engine.base.Engine ()


,id,first_name,last_name,email,phone,address,zip,city,description,created
0,12,Manu,Ginobili,el@manu.ch,None,Bahnhofstrasse,6000,Luzern,None,2004-03-11
1,33,Chauncey,Billups,chauncey@billups.com,None,None,6600,Locarno,D-fense for the culture,2001-05-15
2,215,Mike,Malone,mike@malone.ch,None,Rue des Templiers,1700,Fribourg,None,2005-10-23
3,304,Kyrie,Irving,flat@mindset.com,+41 243 503,None,9000,St Gallen,"Everything is flat, even the books",2014-01-02
4,336,Joe,Johnsson,joe@johnsson.com,None,None,1000,Lausanne,Super cool,1998-04-20


As promised, we obtained a fresh dataframe to play with!

## to_sql

now let's create a new Album column:

In [ ]:
dfnew = df.iloc[-2:,:].copy()

In [ ]:
dfnew

,id,first_name,last_name,email,phone,address,zip,city,description,created
9,889,Shaquille,Oneal,baby@shaq.ch,None,Rue du lac,1200,Geneva,Baby shaq doo doo doo,2001-11-15
10,890,Lisa,Christl,l@gmail.com,+41 220,Munich,9000,Munich,Super Cool,2021-10-08


In [ ]:
dfnew['id'] = [900,901]

In [ ]:
with engine.connect() as con:
    dfnew.to_sql('User', con, if_exists='append', index=False)

2021-05-28 14:47:24,748 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Album")
2021-05-28 14:47:24,751 INFO sqlalchemy.engine.base.Engine ()
2021-05-28 14:47:24,755 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-05-28 14:47:24,759 INFO sqlalchemy.engine.base.Engine INSERT INTO "Album" ("AlbumId", "Title", "ArtistId", "Column1") VALUES (?, ?, ?, ?)
2021-05-28 14:47:24,761 INFO sqlalchemy.engine.base.Engine ((348, 'Mozart: Chamber Music', 274, None), (349, 'Koyaanisqatsi (Soundtrack from the Motion Picture)', 275, None))
2021-05-28 14:47:24,762 INFO sqlalchemy.engine.base.Engine COMMIT
2021-05-28 14:47:24,767 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-05-28 14:47:24,767 INFO sqlalchemy.engine.base.Engine ()
